In [ ]:
from __future__ import print_function
from __future__ import division

""""
This module solves the total variation control problem
  min_{u\in BV(\Omega)} 1/2 \|y-yd\|^2 + beta TV(u)
      s.t.                    Ay = u in \Omega
                               y = 0 on \partial\Omega
using the approach described in the paper
"A path-following inexact Newton method for PDE-constrained optimal control in BV"
by Dominik Hafemeyer and Florian Mannel, see
https://imsc.uni-graz.at/mannel/BVNEWT.pdf.
It uses FEniCS (https://fenicsproject.org/).
"""

__author__ = "Dominik Hafemeyer and Florian Mannel"
__date__   = "November 10, 2020"

import scipy.io as sio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.mlab as mlab
from scipy.sparse import spdiags,identity,bmat,csr_matrix,csc_matrix
from scipy.sparse.linalg import spsolve,LinearOperator,factorized,gmres
from dolfin import parameters,RectangleMesh,Point,FunctionSpace,TestFunction,TrialFunction,DirichletBC,Constant,dx,assemble,as_backend_type,action,dot,inner,grad,Function,Expression,interpolate,plot,sqrt,norm,vertex_to_dof_map

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

parameters['linear_algebra_backend'] = 'Eigen'

#######################################################################  

gr = 0                               # Graphical output? (1: yes, else: no)
err_on = 1                           # Compute (u,y,p)-errors? (1: yes, else: no)
bet = 1e-4                           # total variation parameter beta
gam = Constant(1)
delt = Constant(1)
beta = Constant(1)
beta.assign(Constant(float(bet)))    
    
N = 128                        # NxN uniform triangular grid is used
    
# path-following
gam_fac = 1e2                  # delt = gam_fac*gam
gam.assign(Constant(1e-2))     # starting value for gam
gam_upd = 0.5                  # initial update for gam
gam_upd_min = 0.25             # minimal update for gam
gam_upd_dcf = 0.95             # reduce update factor of (gam,del) by this factor if convergence is fast            
gam_upd_icf = 3./4.            # weight for increasing the update factor of (gam,del) if convergence is slow           
gam_min  = 1e-6                # termination requires gam<gam_min
gam_save = 0                   # last successfull value of gamma
kappa    = 1e-3                # factor for termination criterion

# Computation of (y,p)
#    Newton's method
Nt_maxit  = 25              # max number of Newton steps
F_tol_min = 1e-6            # When ||F(y,p)|| drops below this bound, we decrease (gam,del)
nns_all,nns_use,nns_u = 0,0,0    # total and actually used number of Newton steps for (y,p), total number for u
#    Newton systems (solved via GMRES)
gmres_maxit = 250
gmres_tol   = F_tol_min     # The Newton systems for (y,p) are solved up to this tolerance (once (gam,del) is small enough)
gmres_rstrt = gmres_maxit   # ensures that no restarts are used
#   line search
minstepl     = 1e-4         # minimal allowed step length
sigma_upd    = 0.5          # damping factor
tau          = 1e-4         # fraction of decrease factor

# Computation of u
#    Newton's method
Ntu_maxit = 250             # max number of Newton steps to compute u before u-iteration is terminated
Ntu_safeg = 75              # if, for fixed (gam,del), less than safeg_fac*Ntu_safeg Newton steps are required in total for u,
safeg_fac = 0.75            # then the update factor for (gam,del) is decreased, respectively, increased if more than Ntu_safeg steps are needed.
F_tol_u   = 1e-12           
#   line search
minstepl_u  = 1e-8          # minimal allowed step length
sigma_upd_u = 0.5           # damping factor for line search

# auxiliary functions
def nrm(F):
    F1,F2 = np.split(F,2)
    F1_FE.vector()[:] = F1
    F2_FE.vector()[:] = F2
    return (norm(F1_FE,'L2')**2+norm(F2_FE,'L2')**2)**0.5

def nrm2(F):
    F3_FE.vector()[:] = F
    return norm(F3_FE,'L2')

def nrm3(F):
    F1,F2 = np.split(F,2)
    F1_FE.vector()[:] = F1
    F2_FE.vector()[:] = F2/float(beta)
    return (norm(F1_FE,'H1')**2+norm(F2_FE,'H1')**2)**0.5

def compute_u(nns_u,nrmF,F_tol):
    global num_iters
    global B
    count = 0
    sigma_u = 0.0
    Mp.vector()[:] = MYU*p.vector()
    F_impl = gamL*u.vector() + DTV() - Mp.vector()
    merit_impl_old = 0.0
    merit_impl_new = nrm2(F_impl)
    while count<=Ntu_maxit:
        if abs(merit_impl_new-merit_impl_old)<=1e-2*sigma_u*F_tol_u or nrmF<=F_tol:
            sigma_u = 0.0
            err = merit_impl_old
            break
        merit_impl_old = merit_impl_new
        du.vector()[:] = spsolve(gamL+DDTV(),-F_impl)
        count  += 1
        sigma_u = 1.0
        # line search u
        while sigma_u>=minstepl_u:
            u.vector()[:] += sigma_u*du.vector()[:]
            nrmst = nrm2(sigma_u*du.vector()[:])
            F_impl = gamL*u.vector() + DTV() - Mp.vector()
            merit_impl_new = nrm2(F_impl)
            u.vector()[:] -= sigma_u*du.vector()[:]
            if merit_impl_new<(1+1/count**2)*merit_impl_old - 0.01*min(1,float(gam)**2)*nrmst**2:break
            sigma_u *= sigma_upd_u
        u.vector()[:] += sigma_u*du.vector()[:]
        err = merit_impl_new
        if err<=max(F_tol_u,min(float(gam)**2,1e-2)):break
    nns_u += count
    return err,nns_u

def assemble_csr(a,bc=None):
    A = assemble(a)
    if bc is not None:
        bc.apply(A)
    rows, cols, vals = as_backend_type(A).data()
    return csr_matrix((vals,cols,rows))

def plot_(func,tis):
    fig = plt.figure()
    CS = plot(func,cmap=plt.cm.get_cmap('plasma'))
    cbar = plt.colorbar(CS)
    plt.show()

def println():
    print('--------------------------------------------------------------------------------------')

#Create mesh and define function space
mesh = RectangleMesh(Point(-1.0,-1.0),Point(1.0,1.0),N,N,'left/right')
Uh = FunctionSpace(mesh,'CG',1)
Yh = FunctionSpace(mesh,'CG',1)
NU,NY = Uh.dim(),Yh.dim()
v  = TestFunction(Uh)
w  = TrialFunction(Uh)
vy = TestFunction(Yh)
wy = TrialFunction(Yh)
bc = DirichletBC(Yh,Constant(0.0),lambda x,on_boundary:on_boundary)

# Construct invariant matrices of Newton system
MYY = assemble_csr(wy*vy*dx,bc)
MYU = assemble_csr(wy*v*dx,bc)
MUY = assemble_csr(w*vy*dx)
MUU = assemble_csr(w*v*dx)
#Z = csr_matrix((NY,NY))

# Define problem data and construct exact solution
u,z,Mz = Function(Uh),Function(Yh),Function(Yh)
L = assemble_csr(dot(grad(w),grad(v))*dx) + MUU
A = assemble_csr(dot(grad(wy),grad(vy))*dx,bc)
#Ahatinv = spdiags(1.0/A.diagonal(),0,NU,NU,format='csr')
Q = factorized(csc_matrix(A))
z = interpolate(Expression('''x[0]>-0.5&&x[0]<0.5&&x[1]>-0.5&&x[1]<0.5? 1:0''',degree=5),Yh)
if gr==1: plot_(z,'target')
Mz.vector()[:] = MYY*z.vector()
bc.apply(Mz.vector())

# Initialization
y,p,u = Function(Yh),Function(Yh),Function(Uh)
F1_FE,F2_FE,F3_FE = Function(Yh),Function(Yh),Function(Uh)
y_save,p_save,u_save = Function(Yh),Function(Yh),Function(Uh)
y.vector()[:],p.vector()[:],u.vector()[:] = 0,0,0
y_old,p_old,u_old = Function(Yh),Function(Yh),Function(Uh)
du,dd = Function(Uh),Function(Uh)
MUYu,Mp = Function(Yh),Function(Uh)

# more auxiliary functions
def DTV():
    return assemble((beta/sqrt(delt+dot(grad(u),grad(u)))*dot(grad(u),grad(v)))*dx)

def DDTV():
    return assemble_csr( (beta/sqrt(delt+dot(grad(u),grad(u))) * (dot(grad(w),grad(v)) - (dot(grad(w),grad(u))*dot(grad(u),grad(v))/(delt+dot(grad(u),grad(u))))) )*dx)

def computeJ():
    return assemble((0.5*(y-z)**2+gam/2*(inner(u,u)+dot(grad(u),grad(u)))+beta*(sqrt(delt+dot(grad(u),grad(u)))))*dx)

def computeF(nns_u,nrmF,F_tol):
    err,nns_u = compute_u(nns_u,nrmF,F_tol)
    MUYu.vector()[:] = MUY*u.vector()
    bc.apply(MUYu.vector())
    F1 = A*p.vector() + MYY*y.vector() - Mz.vector()
    F2 = A*y.vector() - MUYu.vector()
    return np.hstack([F1,F2]),err,nns_u

def mvu(v):
    global B
    return B.solve(v)          

def mvyp(v):
    vy,vp = np.split(v,2)
    vpq = Q(vp)
    return np.hstack([vpq,Q(vy-MYY*vpq)])

def mv(v):
    vy,vp = np.split(v,2)
    dd.vector()[:] = Prec(MYU*vp)
    MUYu.vector()[:] = MUY*dd.vector()
    bc.apply(MUYu.vector())
    return np.hstack([MYY*vy + A*vp, A*vy - MUYu.vector()])

# path-following Newton method [=main routine]
println(),println()
print('Solving for beta = %1.3e' % (beta))
println()
delt.assign(Constant(gam_fac*gam))
rate_gam = 0
converged = 0
it_mon = 1
err_u = 0
r,r_old,r_u,r_u_old = kappa+1,kappa+1,kappa+1,kappa+1
err_new,err_old = 1,1
#precond_dF = bmat([[Z,Ahatinv],[Ahatinv,-Ahatinv*MYY*Ahatinv]],format='csr')   # alternative preconditioner
precond_dF = LinearOperator((2*NY,2*NY),matvec=mvyp)
while not converged: 
    println()
    if gam_save != 0:
        rate_gam = float(gam)/gam_save
    else: 
        rate_gam = 0.      
    print('gamma = %1.2e delta = %1.2e rate = %1.2f' % (float(gam),float(delt),rate_gam))
    println()
    F_tol = max(F_tol_min,float(gam))
    nns_u_st = nns_u
    gamL = float(gam)*L
    for it in range(Nt_maxit+1):
        sigma_min = 1
        if it==0:
            print('It %2d: XXXXXXXXXXXXX ' %(it),end='')
            F,err_u,nns_u = computeF(nns_u,2*F_tol,F_tol)
            nrmF,J = nrm(F),computeJ()
            print('it_u=%3d |F(u)|=%1.2e XXXXXXXXXXX |F(y,p)|=%1.2e J=%1.2e\n' % (nns_u-nns_u_st,err_u,nrmF,J),end='')
        if nrmF<=F_tol: break 
        print('It %2d: ' %(it+1),end='')
        Prec = factorized(csc_matrix(gamL+DDTV()))
        dF = LinearOperator((2*NY,2*NY),matvec=mv)
        tol = gmres_tol
        deltx,info = gmres(dF,-F,tol=tol,atol=tol,maxiter=gmres_maxit,M=precond_dF,restart=gmres_rstrt)
        if info!=0 : print('\x1b[6;30;42m' + '!!!' + '\x1b[0m',end=' ')   # print warning if gmres does not terminate successfully
        print('|dx|=%1.2e ' % nrm(deltx),end='')
        dy,dp = np.split(deltx,2)
        # line search (y,p)
        sigma = 1.0      
        merit_old = nrmF
        nns_u_sti = nns_u
        while sigma>=minstepl:
            y.vector()[:] += sigma*dy
            p.vector()[:] += sigma*dp
            nrmst = sigma*nrm(deltx)
            F,err_u,nns_u = computeF(nns_u,nrmF,F_tol)
            nrmF,J = nrm(F),computeJ()
            merit_new = nrmF
            y.vector()[:] -= sigma*dy
            p.vector()[:] -= sigma*dp  
            if merit_new<=(1+1/(it_mon**2))*merit_old - tau*nrmst**2:break
            sigma *= sigma_upd
        if sigma<minstepl:
            print('it_u=%3d |F(u)|=%1.2e sig=%1.1e!' %(nns_u-nns_u_sti,err_u,sigma),end='')
        else:
            print('it_u=%3d |F(u)|=%1.2e sig=%1.1e ' %(nns_u-nns_u_sti,err_u,sigma),end='')
        print('|F(y,p)|=%1.2e J=%1.2e\n' % (nrmF,J),end='')
        y.vector()[:] += sigma*dy
        p.vector()[:] += sigma*dp
        it_mon += 1
    nns_all += it
    nns_use += it

    # check convergence
    if nrmF<=F_tol:               # converged: accept iterate 
        if gr==1: plot_(u,'control')
        r_old = r
        r_u_old = r_u
        r = nrm3(np.hstack([y_save.vector()-y.vector(),p_save.vector()-p.vector()])) 
        r_u = nrm2(u_save.vector()-u.vector())
        print('r_(y,p) = %1.2e  r_u = %1.2e' % (r,r_u))
        println()
        y_save.vector()[:] = y.vector()
        p_save.vector()[:] = p.vector()
        u_save.vector()[:] = u.vector()
        gam_old  = gam_save
        gam_save = float(gam)
        if nns_u-nns_u_st<=safeg_fac*Ntu_safeg:     # (very) few Newton steps needed for u: decrease (much) faster
            if nns_u-nns_u_st<=0.5*safeg_fac*Ntu_safeg: gam_upd *= gam_upd_dcf**2
            else: gam_upd *= gam_upd_dcf
        if nns_u-nns_u_st>Ntu_safeg:                # many Newton steps needed for u: decrease slower
            gam_upd = gam_upd_icf*gam_upd+(1-gam_upd_icf)        
        gam_upd_old = gam_upd   
    else:                         # not converged: restore last successful iterate and try again with larger gam
        nns_use -= it
        gam.assign(Constant(gam_save))
        gam_upd = gam_upd_dcf*gam_upd+(1-gam_upd_dcf)
        y.vector()[:] = y_save.vector()
        p.vector()[:] = p_save.vector()
        u.vector()[:] = u_save.vector()
        print('\nIterate rejected: warm start with larger gamma')
    tt = nrm3(np.hstack([y.vector(),p.vector()]))
    if float(gam)<=gam_min and nrmF<=F_tol and r<=(1-gam_upd)*kappa*tt and r_old<=(1-gam_upd)*kappa*tt:
        converged = 1
        gam_upd_old = gam_upd
        gam_upd = 1.0
    if gam_upd<gam_upd_min: gam_upd = gam_upd_min
    gam.assign(Constant(gam*gam_upd))
    delt.assign(Constant(gam_fac*gam))
    it_mon = 1

# Iteration finished -- evaluate results
println(),println()
print('Final value of gamma/delta: %1.3e/%1.3e' %(float(gam),float(delt)))
print('Number of Newton steps (y,p)/u: %1.3d/%1.3d' %(nns_all,nns_u))
if err_on==1 : 
    #load reference solution and create grid&function spaces for it
    meshf = RectangleMesh(Point(-1.0,-1.0),Point(1.0,1.0),1024,1024,'left/right')
    Uhf = FunctionSpace(meshf,'CG',1)
    Yhf = FunctionSpace(meshf,'CG',1)
    NUf,NYf = Uhf.dim(),Yhf.dim()
    v2df  = vertex_to_dof_map(Uhf)
    v2dYf = vertex_to_dof_map(Yhf)
    exact = sio.loadmat('BVNEWT_refsol')
    u_ex = Function(Uhf)
    u_vec = exact['u']
    u_vec = u_vec[(0)]
    u_ex.vector()[v2df] = u_vec
    y_ex = Function(Yhf)
    y_vec = exact['y']
    y_vec = y_vec[(0)]
    y_ex.vector()[v2dYf] = y_vec
    p_ex = Function(Yhf)
    p_vec = exact['p']
    p_vec = p_vec[(0)]
    p_ex.vector()[v2dYf] = p_vec
    J_ex = exact['J']
    J_ex = J_ex[(0)]     
    F4_FE,F5_FE = Function(Uhf),Function(Yhf)
    err_J = np.abs(J-J_ex)
    uf = interpolate(u,Uhf)
    F4_FE.vector()[:] = uf.vector() - u_ex.vector()
    err_u_L1 = assemble(abs(F4_FE)*dx)
    err_u_L2 = norm(F4_FE,'L2')
    err_u_H1 = norm(F4_FE,'H1')
    yf = interpolate(y,Yhf)
    F5_FE.vector()[:] = yf.vector() - y_ex.vector()
    err_y_L1 = assemble(abs(F5_FE)*dx)
    err_y_L2 = norm(F5_FE,'L2')
    err_y_H1 = norm(F5_FE,'H1')
    pf = interpolate(p,Yhf)
    F5_FE.vector()[:] = pf.vector() - p_ex.vector()
    err_p_L1 = assemble(abs(F5_FE)*dx)
    err_p_L2 = norm(F5_FE,'L2')
    err_p_H1 = norm(F5_FE,'H1')
    print('|J_h-J| = %1.3e' %err_J)
    print('|u_h-u|_{L^1} = %1.2e,   |y_h-y|_{L^1} = %1.2e,   |p_h-p|_{L^1} = %1.2e' %(err_u_L1,err_y_L1,err_p_L1))
    print('|u_h-u|_{L^2} = %1.2e,   |y_h-y|_{L^2} = %1.2e,   |p_h-p|_{L^2} = %1.2e' %(err_u_L2,err_y_L2,err_p_L2))
    print('|u_h-u|_{H^1} = %1.2e,   |y_h-y|_{H^1} = %1.2e,   |p_h-p|_{H^1} = %1.2e' %(err_u_H1,err_y_H1,err_p_H1))

# Plot solution
if gr==1: 
    plot_(u,'u') #plot_(p,'p'),plot_(y,'y'),
    if err_on==1:plot_(u_ex,'u_ex'),plot_(F4_FE,'u_error') #plot_(p_ex,'p_ex'),plot_(F5_FE,'p_error'),